## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-02-23-47-20 +0000,nyt,FDA Approves Generic Version of Abortion Pill ...,https://www.nytimes.com/2025/10/02/health/abor...
1,2025-10-02-23-41-59 +0000,bbc,Trump wields axe over 'Democrat Agencies' as b...,https://www.bbc.com/news/articles/ckg201rnw2zo...
2,2025-10-02-23-40-54 +0000,startribune,Omar Fateh campaign fined for claiming DFL end...,https://www.startribune.com/omar-fateh-campaig...
3,2025-10-02-23-34-25 +0000,nyt,Trump Asked Universities to Sign a Compact. So...,https://www.nytimes.com/2025/10/02/us/trump-un...
4,2025-10-02-23-32-17 +0000,nyt,Trump ‘Determined’ the U.S. Is Now in a War Wi...,https://www.nytimes.com/2025/10/02/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2504/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,34
172,shutdown,23
43,manchester,15
42,attack,15
40,synagogue,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
28,2025-10-02-22-15-00 +0000,wsj,President Trump intensified pressure on Democr...,https://www.wsj.com/politics/trump-to-meet-wit...,116
21,2025-10-02-22-55-39 +0000,nypost,Trump plans for ‘thousands’ of federal firings...,https://nypost.com/2025/10/02/us-news/trump-sh...,84
216,2025-10-02-13-17-00 +0000,wsj,Office of Management and Budget Director Russe...,https://www.wsj.com/politics/policy/russell-vo...,80
322,2025-10-02-00-46-00 +0000,wsj,Congress’s decision about what to do about exp...,https://www.wsj.com/politics/policy/government...,80
290,2025-10-02-05-15-26 +0000,nypost,Trump admin to provide Ukraine with intelligen...,https://nypost.com/2025/10/02/us-news/trump-ad...,77


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
28,116,2025-10-02-22-15-00 +0000,wsj,President Trump intensified pressure on Democr...,https://www.wsj.com/politics/trump-to-meet-wit...
253,73,2025-10-02-10-09-41 +0000,cbc,U.K. police say deadly rampage at Manchester s...,https://www.cbc.ca/news/world/uk-synagogue-inc...
55,48,2025-10-02-21-25-48 +0000,nypost,Ukraine pitches plan to help US sell American ...,https://nypost.com/2025/10/02/world-news/ukrai...
322,45,2025-10-02-00-46-00 +0000,wsj,Congress’s decision about what to do about exp...,https://www.wsj.com/politics/policy/government...
210,42,2025-10-02-13-39-46 +0000,wapo,"Israel intercepts Gaza aid flotilla, detains T...",https://www.washingtonpost.com/world/2025/10/0...
320,41,2025-10-02-01-03-38 +0000,latimes,Trump administration says it will cut $8 billi...,https://www.latimes.com/environment/story/2025...
259,39,2025-10-02-10-00-00 +0000,nypost,NYC’s digital members-only clubs are harnessin...,https://nypost.com/2025/10/02/business/soho-ho...
64,36,2025-10-02-21-07-00 +0000,wsj,Senators on both sides of the aisle pressed th...,https://www.wsj.com/politics/national-security...
326,36,2025-10-02-00-30-00 +0000,wsj,The Court kept Lisa Cook in her job until it c...,https://www.wsj.com/politics/policy/the-suprem...
294,34,2025-10-02-05-01-25 +0000,nypost,UK university student dies by suicide 3 months...,https://nypost.com/2025/10/02/world-news/scotl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
